### Введение
Изначально появилась такая задача: локализовать точку на плоскости. В практическом применении это выглядит примерно так: мы получаем каким-то образом свои координаты, и по ним требуется определить местоположение на карте. Так как карта заранее известна, ее можно предподсчитать, чтобы потом время обработки запроса было как можно меньше. Но карты обычно довольно большие, так что предподсчет должен использовать как можно меньше дополнительной памяти. Трапецоидная карта в какой-то мере позволяет решить эту задачу, используя в среднем $O(n)$ памяти, $O(nlog(n))$ времени на предподсчет и $O(log(n))$ времени на запрос.

### Трапецоидная карта
Изначально имеется множество непересекающихся отрезков, лежащих на плоскости. Это множество "условно" помещается в bounding box $R$ – прямоугольник, ограничивающий все отрезки. Также условимся, что среди всех вершин отрезков никакие две различные вершины не лежат на одной вертикальной прямой (но при этом они могут совпадать). В дальнейшем эти ограничения можно будет снять.

Трапецоидная карта – структура данных для локализации точки среди этих отрезков. Она получается следующим путем – из каждой вершины выпускается два вертикальных луча, вверх и вниз, до первого пересечения с другим отрезком или с $R$.

![Пример трапецоидной карты](images/tmap_example.jpg)

Рассмотрим, что из себя представляет отдельно каждый трапецоид:

<details>
  <summary><b>Лемма. Любой трапецоид ограничен одним или двумя вертикальными и двумя не вертикальными отрезками</b></summary>
  <p style="margin:20px"> Обозначим наш трапецоид $f$. Для начала покажем, что $f$ – выпуклый. Любая угловая точка у $f$ по построению является либо концом отрезка, либо пересечением вертикального луча с другим отрезком или с $R$, либо это один из углов $R$. Так как лучи вертикальные, то угол пересечения с отрезком не превзойдет 180 градусов, а угол с участием $R$ равен 90 градусам. Следовательно, $f$ – выпуклый, так как вертикальные лучи устраняют все невыпуклости.</p>
  <p style="margin:20px"> Из построения $f$ следует, что должно быть не менее 2 не вертикальных отрезков. Допустим, их будет больше 2. Тогда как минимум 2 из них будут смежными с верхней (или нижней) стороны, и у них будет общая точка. Но в ней должен был быть вертикальный луч, который создал бы дополнительный трапецоид! Значит, не вертикальных отрезков ровно два. Также по построению $f$ не может иметь больше 2 вертикальных отрезков, но один из них может отсутствовать (верхний и нижний отрезок имеют одну общую точку, а трапецоид будет треугольником).</p>
</details>
   
Отсюда и берется название трапецоидных карт, так как трапецоид представляет из себя либо трапецию, либо треугольник.

Обозначим отрезок, лежащий сверху трапецоида, $top(\Delta)$, и аналогично лежащий снизу $bottom(\Delta)$. Рассмотрим возможные варианты расположения левого отрезка:
- он отсутствует, вместо него точка пересечения $top(\Delta)$ и $bottom(\Delta)$;
- он образован лучом, идущим вниз из левой точки $top(\Delta)$;
- он образован лучом, идущим вверх из левой точки $bottom(\Delta)$;
- он образован двумя лучами из правой точки отрезка, лежащего слева от $\Delta$;
- это левая граница $R$.

![Случаи расположения leftp](images/leftp_cases.jpg)

В каждом случае (за исключением пятого) левый отрезок определяла одна точка $p$ – вершина одного из отрезков. Обозначим ее как $leftp(\Delta)$ (в случае для $R$ это будет $None$, так как координата его левой нижней точки не известна, да и сам $R$ нереален). Аналогичным способом получим 5 возможных случаев для расположения правого отрезка, обозначив правую вершину как $rightp(\Delta)$. Заметим, что трапецоид однозначно задается этим набором: $top(\Delta)$, $bottom(\Delta)$, $leftp(\Delta)$, $rightp(\Delta)$.

А сколько же всего трапецоидов мы получим?
<details>
  <summary><b>Лемма. Трапецоидная карта, построенная на $n$ отрезках, содержит не более $6n+4$ вершин и $3n+1$ трапецоидов</b></summary>
  <p style="margin:20px">Вершиной трапецоида может являться либо "условная" вершина $R$ (всего их четыре), либо конец отрезка ($2n$ вершин), либо конец вертикального луча, выходящего из конца отрезка ($2n*2=4n$ вершин) – итого не более $6n+4$ вершин.</p>
  <p style="margin:20px">Для ограничения числа трапецоидов рассмотрим точку $leftp(\Delta)$. Она является либо концом отрезка, либо $None$. Если вершина $None$, то этот трапецоид ограничен левой стороной $R$, а такой трапецоид будет один. Правый конец каждого отрезка задает не более одной $leftp(\Delta)$ ($4$ случай из примера выше); левый конец отрезка – не более двух $leftp(\Delta)$ ($2-3$ случаи выше. Однако в $1$ случае, когда $k>1$ отрезков имеют общую левую точку, $leftp(\Delta)$ может быть общей сразу для $k+1$ трапецоидов. В этом случае условимся, что каждый из $k$ отрезков задает $leftp(\Delta)$ лишь для верхнего и нижнего трапецоида, тогда каждому трапецоиду соответствует два отрезка и все хорошо). Значит, каждая точка $leftp(\Delta)$ задает не более трех трапецоидов, а общее количество трапецоидов не превышает $3n+1$.</p>
</details>

В дальнейшем нам понадобятся трапецоиды с общей вертикальной прямой – назовем их соседями. По нашему условию на одной вертикальной прямой лежит максимум одна вершина, значит трапецоид может иметь до двух соседних трапецоидов слева и справа (иначе бы могло быть любое число соседей). На примере выше в первом и пятом случаях у трапецоида левые соседи отсутствую, во втором присутствует только нижний, в третьем только верхний, а в четвертом сразу оба соседа.

### Локализация точки на трапецоидной карте
Рассмотрим процесс локализации точки $q$ на трапецоидной карте $J$, приведенной ниже.

![Пример локализации в J](images/tmap_localization_example.jpg)

Для быстрой локализации точки заведем локализационную структуру $D$. Она представляет из себя ациклический ориентированный граф с одним корнем (проще говоря, дерево), в котором листами являются трапецоиды. <b>Важное замечание:</b> каждому трапецоиду в графе соответствует ровно один лист.

Все узлы графа делятся на 2 типа:

- $X$ – соответствует вершине отрезка ($p_i$ – левая вершина, а $q_i$ – правая)
- $Y$ – соответствует самому отрезку $s_i$

У каждого узла графа 2 исходящих ребра. При запросе локализации точки $q$ на трапецоидной карте $J$ мы спускаемся по графу от корня к нужному трапецоиду. В случае узла $X$ мы сравниваем вершины лексикографически: если $q$ меньше, то идем по синему ребру, иначе по оранжевому. В случае узла типа $Y$ мы проверяем, лежит ли $q$ выше или ниже отрезка. Если $q$ выше, то идем по синему ребру, иначе по оранжевому. В конце мы доходим до соответствующего точке $q$ трапецоида.

![Пример локализации в D](images/localization.gif)

Также могут быть случаи, когда $q$ совпадает с вершиной или принадлежит отрезку. Нельзя не признать, что $q$ все ещё принадлежит трапецоиду (причем, сразу нескольким), и при выборе любого исходящего из текущего узла ребра узла мы придем к нужному трапецоиду. Но нельзя не согласиться, что вершина или отрезок более точно локализуют точку запроса, нежели трапецоид, значит мы можем выдать в качестве ответа отрезок, на котором лежит $q$ (или в случае вершины отрезок, который ей соответствует).

### Построение поисковой структуры и трапецоидной карты
Изначально карта $J_0$ состоит из единственного трапецоида, у которого отсутствуют и соседи, и верхний/нижний отрезок, и левая/правая точка (фактически, весь $R$), и к тому же он лежит в корне $D$. Алгоритм построения трапецоидной карты инкрементальный: в уже существующую карту по одному добавляются новые отрезки. Что необходимо сделать при добавлении очередного отрезка $s_i$:
- найти трапецоиды $\Delta_0,\Delta_1,...,\Delta_k$, которые пересекает $s_i$;
- удалить их из $J_{i-1}$ и заменить на новые трапецоиды, появившиеся при вставке $s_i$
- заменяем листы из $D_{i-1}$, соответствующие старым трапецоидам, на новые

При добавлении $s_i$ необходимо найти $\Delta_0,\Delta_1,...,\Delta_k$. Для этого мы находим $\Delta_0$, локализуя левую точку отрезка $s$. Далее легко получить $\Delta_1,...,\Delta_k$, проходя вправо по соседям. Для проверки, верхним или нижним будет следующий трапецоид, нужно проверить поворот точки $rightp(\Delta_j)$ относительно прямой $s_i$. Поиск остановится, когда правая вершина $s_i$ окажется левее $rightp(\Delta_k)$. Таким образом, получим $\Delta_0,\Delta_1,...,\Delta_k$ за $O(h+k)$, где $h$ – высота $D$.

Далее необходимо удалить старые трапецоиды и на их место вставить новые. Сначала разберем простой случай, когда $s_i$ целиком попал в один трапецоид. $s_i$ вместо $\Delta$ порождает 4 новых трапецоида. Необходимо обновить указатели у соседей $\Delta$ и новых трапецоидов, а также заменить в $D_{i-1}$ лист, соответствовавший $\Delta$, на новое поддерево высоты 3, как показано на рисунке ниже. Этот случай обрабатывается за $O(1)$.

![Один трапецоид](images/single_s_i.jpg)

Теперь рассмотрим сложный случай, когда $s_i$ пересекает $\Delta_0,\Delta_1,...,\Delta_k$. Необходимо по-разному обработать 3 случая:
- левый конец отрезка внутри $\Delta_0$
- правый конец отрезка внутри $\Delta_k$
- отрезок полностью пересекает трапецоид

В первых двух случаях концы $s_i$ порождают новые вертикальные лучи, то есть необходимо разбить $\Delta_0$ и/или $\Delta_k$ на три трапецоида. Кроме того, $s_i$ пересечет некоторые другие вертикальные лучи, значит надо подразбить трапецоиды вдоль $s_i$, начиная с $\Delta_0$. При переходе от $\Delta_i$ к $\Delta_{i+1}$ смотрим, с какой стороны от $s_i$ лежит точка $rightp(\Delta_i)$. Если она лежит сверху, то трапецоид ниже $s_i$ продолжится вдоль отрезка, а трапецоид сверху $s_i$ закончится. Выставляем корректно все вершины и всех соседей для трапецоидов, за исключением того, что у нижнего трапецоида $rightp(\Delta_{low})=null$. Запомним его для правильной расстановки указателей на следующем шаге. Мы сможем выставить ему $rightp$, когда появится такой трапецоид $\Delta_j$, что $rightp(\Delta_j)$ будет ниже $s_i$, или мы дойдем до конца отрезка $s_i$. С трапецоидами, тянущимися сверху, поступаем аналогично.

В $D_{i}$ листы, соответствовавшие $\Delta_0,\Delta_1,...,\Delta_k$, заменяются на новые поддеревья. В первом случае лист $\Delta_0$ заменяется на поддерево высотой 3 с узлом типа $X$ и $Y$, которые указывают на 3 новых трапецоида. Во втором случае аналогично, но только для правого конца $s_i$. В третьем случае лист заменится на поддерево высоты 2 и узел типа $Y$, указывающий на два новых трапецоида. Все это делается за $O(k)$, а высота дерева увеличивается не более, чем на 2, что видно на рисунке ниже.

![Несколько трапецоидов](images/multi_s_i.jpg)

Не помешает рассмотреть пример вставки очередного отрезка на карту, приведенную выше.

![Вставка в J](images/map_insert.gif)

Как видно, новый отрезок $s_{17}$ пересекает 4 трапецоида. Вершины отрезка попадают внутрь трапецоидов $\Delta_0$ и $\Delta_{14}$, поэтому в новом дереве вместо этих трапецоидов сначала вставляются $p_{17}$ и $q_{17}$ (выделены желтым), а уже затем сам отрезок (как и для двух других трапецоидов $\Delta_{5}$ и $\Delta_{10}$).

![Вставка в D](images/tree_insert.gif)

В сумме локализация и вставка нового отрезка $s_i$ займут $O(h+k)$ времени. Таким образом мы получим корректную трапецоидную карту $J$ и поисковую структуру $D$, так как на каждом шаге добавление нового отрезка было корректным.

### Асимптотика и память
Порядок добавления отрезков очень важен: в случае попадания нового отрезка целиком в трапецоид высота $D$ увеличится на 3, а в худшем случае высота дерева может составить $3n$. Тогда алгоритм будет строить дерево за $O(n^2)$, а локализация точки будет выполняться за $O(n)$. 
<details>
  <summary>Несложно задать такую последовательность отрезков.</summary>
  <p style="margin:20px">Каждый следующий отрезок должен целиком попадать внутрь верхнего/нижнего трапецоида, созданного на предыдущем шаге. </p>
</details>

Для сглаживания этой неприятности отрезки добавляют в случайном порядке, что дает более приемлемую временную оценку.

Зафиксируем множество $S$ из $n$ отрезков и точку запроса $q$. Всего возможно $n!$ перестановок отрезков, а значит $n!$ различных структур $D$. В этом случае мы можем оценить ожидаемое значение высоты $D$. Добавим немножко теорвера: обозначим количество узлов на пути локализации $q$, созданных на $i$-ой итерации алгоритма, за $x_i$ – это случайная величина. Найдем матожидание длины пути:

$E\left[\sum_{i=1}^{n}{x_i}\right]=\sum_{i=1}^n{E\left[x_i\right]}$

Также мы знаем, что $x_i\leq3$. Обозначим $p_i$ как вероятность встретить на пути локализации $q$ узел, созданный на $i$-ой итерации. Ясно, что $E\left[x_i\right]\leq3p_i$. Оценим $p_i$. Необходимо понять, когда $p_i\neq0$. Это так только в том случае, когда $q \in \Delta_{i-1}$, но на шаге $i$ трапецоид $\Delta_{i-1}$ был удален, а точка $q$ перешла в трапецоид $\Delta_i$. Применим так называемый "backwards-analysis": на $i$ шаге удалим случайный отрезок $s_k$ : $k \in [1,i]$ и оценим вероятность исчезновения трапецоида $\Delta_i$. Это произойдет в 4 случаях:
- $top(\Delta_i)=s^{'}_i$
- $bottom(\Delta_i)=s^{'}_i$
- $leftp(\Delta_i)$ – конец отрезка $s^{'}_i$
- $rightp(\Delta_i)$ – конец отрезка $s^{'}_i$

Так как $s_i$ вставлялись в случайном порядке, то для каждого случая вероятность того, что $s^{'}_i=s_i$ равна $\frac{1}{i}$, а в сумме она не превосходит $\frac{4}{i}$. Таким образом $\sum_{i=1}^n{E\left[x_i\right]} \leq \sum_{i=1}^n{3p_i} \leq \sum_{i=1}^n{\frac{12}{i}} = 12\sum_{i=1}^n{\frac{1}{i}} = 12H_n$, где $H_n$ – гармонический ряд, который асимптотически равен $12ln(n) \approx O(log(n))$. Значит, ожидаемое время локализации составит $O(log(n))$.

Теперь вернемся к размеру $D$. В худшем случае на каждой итерации алгоритма новый отрезок будет пересекать все трапецоиды, и тогда размер структуры составит $O(n^2)$. Найдем ожидаемый объем памяти. Мы знаем, что в конце алгоритма у нас будет $O(n)$ листов в $D$, тогда размер $D$ составит $O(n) + \sum_{i=1}^{n}{E\left[x_i-1\right]} = O(n) + \sum_{i=1}^{n}{E\left[x_i\right]}$, где $x_i$ – количество трапецоидов, созданных на $i$-ой итерации. Надо ограничить $E\left[x_i\right]$.

Снова применим backwards-analysis. Зафиксируем набор отрезков $S_i$. Введем новую функцию:

$\delta(\Delta, s) = \begin{cases}
1, &\text{если }\Delta\text{ исчезнет при удалении }s_i\\
0, &\text{иначе}
\end{cases}$

$\Delta$ может исчезнуть при удалении $top(\Delta)$, $bottom(\Delta)$, точки $leftp(\Delta)$ или $rightp(\Delta)$ (если они присутствуют у $\Delta$). Значит, на $\Delta$ влияет не более 4 отрезков, тогда $\sum_{s \in S_i}{\sum_{\Delta \in J_i} {\delta(\Delta, s)}} \leq 4 \left|J_i\right| = O(i)$. С учетом того, что вероятность удаления отрезка равна $\frac{1}{i}$, найдем матожидание: $E\left[x_i\right] = \frac{1}{i}\sum_{s \in S_i}{\sum_{\Delta \in J_i}{\delta(\Delta, s)}} \leq \frac{O(i)}{i} = O(1)$. Таким образом, за одну итерацию объем памяти увеличится на $O(1)$, а всего структура займет $O(n)$ памяти.

Остается только получить время работы алгоритма, что довольно просто. Уже известно, что добавление отрезка занимает $O(h+k)$, но ожидаемое значение составляет $O(log(n) + 1) \approx O(log(n))$, а для $n$ отрезков потребуется $O(nlog(n))$ времени.

### Вырожденные случаи

Ранее мы условились, что среди всех вершин отрезков любые две вершины не лежат на одной вертикальной прямой (но при этом они могут совпадать). Разрешим эти случаи. Для этого надо "слегка" повернуть систему координат. При достаточно малом угле поворота никакие две точки не будут лежать на одной вертикальной прямой, и все будет хорошо. Фактически, достаточно упорядочить точки лексикографически: будем считать, что точка $p(x,y_p)$ лежит правее точки $q(x,y_q)$, если $y_q<y_p$, и наоборот.

Более формально, нам необходимо выполнить следующее преобразование для всех точек отрезков:

$\phi : \left( \begin{array}{c} x\\ y\end{array}\right) 
\rightarrow
\left( \begin{array}{c} x + \epsilon y\\ y\end{array}\right)$

Так мы отобразим вертикальную прямую на прямую с углом наклона $\frac{1}{\epsilon}$. Если $\epsilon$ достаточно мал, то это преобразование не поменяет исходный порядок точек по координате $x$.  Конечно, появятся новые вырожденные трапецоиды, которые не могли возникнуть в исходной система координат. Но так как количество отрезков не изменилось, время работы алгоритма останется тем же.

На практике достаточно упорядочить точки лексикографически, и тогда мы сможем добавлять как вертикальные отрезки, так и отрезки с вершинами, лежащими на одной вертикальной прямой.